# Quick Start for You First AI Project using SAP AI Core
Author: https://github.com/dhrubpaul

The steps are analogous with the tutorial: https://developers.sap.com/tutorials/ai-core-helloworld.html
Please open the tutorial and this notebook side-by-side for better understanding.

## Pre-requisite 
Create connection with SAP AI Core. Edit the below cell

In [1]:
%pip install ai-core-sdk

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import yaml
# Load Library
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

# Load environment variables from credentials.yaml
with open('credentials.yaml', 'r') as file:
    env_vars = yaml.safe_load(file)

os.environ['AICORE_CLIENT_ID'] = env_vars['AICORE_CLIENT_ID']
os.environ['AICORE_CLIENT_SECRET'] = env_vars['AICORE_CLIENT_SECRET']
os.environ['AICORE_RESOURCE_GROUP'] = env_vars['AICORE_RESOURCE_GROUP']
os.environ['AICORE_AUTH_URL'] = env_vars['AICORE_AUTH_URL']
os.environ['AICORE_BASE_URL'] = env_vars['AICORE_BASE_URL']

# Create Connection using environment variables
ai_core_client = AICoreV2Client(
    base_url=os.environ['AICORE_BASE_URL'] + "/v2",
    auth_url=os.environ['AICORE_AUTH_URL'] + "/oauth/token",
    client_id=os.environ['AICORE_CLIENT_ID'],
    client_secret=os.environ['AICORE_CLIENT_SECRET']
)
# no output is expected

## Step 1: Create a GitHub repository

*Create a repository on GitHub.com . You may use any your own Git hosting.* Refere step 1 of the tutorial: https://developers.sap.com/tutorials/ai-core-helloworld.html#943ffe1c-b32f-4795-8024-3356466a3f75


## Step 2: Generate a GitHub personal access token

*Perform this step over GitHub*. Refer step 2 of the tutorial: https://developers.sap.com/tutorials/ai-core-helloworld.html#06fb98d7-9efa-427d-aed8-864b56b72f3c

## Step 3: Onboard GitHub to SAP AI Core

In [6]:
import os
import yaml

# Load environment variables from credentials.yaml
with open('credentials.yaml', 'r') as file:
    env_vars = yaml.safe_load(file)

os.environ['ACCESS_TOKEN_NAME'] = env_vars['ACCESS_TOKEN_NAME']
os.environ['REPO_URL'] = env_vars['REPO_URL']
os.environ['GITHUB_USERNAME'] = env_vars['GITHUB_USERNAME']
os.environ['ACCESS_TOKEN_PWD'] = env_vars['ACCESS_TOKEN_PWD']

# on-board new GitHub repository
ai_core_client.repositories.create(
    name=os.environ['ACCESS_TOKEN_NAME'],
    url=os.environ['REPO_URL'],
    username=os.environ['GITHUB_USERNAME'],
    password=os.environ['ACCESS_TOKEN_PWD']
)

**If Error**
- check keys in the top cell
- ensure your GitHub is not blocked by firewall. You repostiory can be private.

In [7]:
# check on-boarding status
response = ai_core_client.repositories.query()
#
for repository in response.resources:
    print('Name:', repository.name)
    print('URL:', repository.url)
    print('Status:', repository.status)


Name: mfalconaro-dev-access-token
URL: https://github.com/matiasfalconaro/sap-ai-core-pipeline
Status: RepositoryStatus.COMPLETED
Name: ncellini-aicore-pipelines
URL: https://github.com/NCellini/aicore-pipelines
Status: RepositoryStatus.COMPLETED
Name: gsalomone-tf-gpu
URL: https://github.com/gianfranco-s/SAP-aicore-gpu
Status: RepositoryStatus.COMPLETED
Name: tomaspf-aicore
URL: https://github.com/tomaspf9/SAP-aicore
Status: RepositoryStatus.COMPLETED


## Step 4: Place a workflow in GitHub

*Perform this step on your GitHub*. Refer step 4 of the tutorial: https://developers.sap.com/tutorials/ai-core-helloworld.html#65c2faa9-0e67-4fee-b639-65a98139911a

## Step 5: Create an application to sync workflows with SAP AI Core

In [13]:
import os
import yaml

# Load environment variables from credentials.yaml
with open('credentials.yaml', 'r') as file:
    env_vars = yaml.safe_load(file)

os.environ['APPLICATION_NAME'] = env_vars['APPLICATION_NAME']
os.environ['REPO_URL'] = env_vars['REPO_URL']
os.environ['PATH'] = env_vars['PATH']
os.environ['REVISION'] = env_vars['REVISION']

ai_core_client.applications.create(
    application_name=os.environ['APPLICATION_NAME'],
    repository_url=os.environ['REPO_URL'],
    path=os.environ['PATH'],
    revision=os.environ['REVISION'],
)

List your applications created for syncing workflows

In [14]:
response = ai_core_client.applications.query()

for app in response.resources:
    print(app.__dict__)


{'path': 'tf-text-classifier', 'revision': 'HEAD', 'repository_url': 'https://github.com/gianfranco-s/SAP-aicore-gpu', 'application_name': 'gsalomone-tf-app'}
{'path': 'LearningScenarios', 'revision': 'HEAD', 'repository_url': 'https://github.com/NCellini/aicore-pipelines', 'application_name': 'learning-scenarios-app-ncellini'}
{'path': '2_hello_world', 'revision': '15_create_an_application_to_sync_workflow', 'repository_url': 'https://github.com/matiasfalconaro/sap-ai-core-pipeline', 'application_name': 'mfalconaro-learning-scenarios-app'}
{'path': 'LearningScenarios', 'revision': 'HEAD', 'repository_url': 'https://github.com/tomaspf9/SAP-aicore', 'application_name': 'tomaspf-learningscenarios'}


## Step 6: Check for errors

In [15]:
response = ai_core_client.applications.get_status(application_name=os.environ['APPLICATION_NAME'])

print(response.message)
for workflow_sync_status in response.sync_ressources_status:
    print(workflow_sync_status.__dict__)


successfully synced (all tasks run)
{'name': 'mfalconaro-first-pipeline', 'kind': 'WorkflowTemplate', 'status': 'Synced', 'message': 'workflowtemplate.argoproj.io/mfalconaro-first-pipeline created'}


## Step 7: Check your scenario

In [26]:
response = ai_core_client.scenario.query(resource_group=os.environ['AICORE_RESOURCE_GROUP'])

for scenario in response.resources:
    print(scenario.__dict__)


{'id': 'tf-text-clf', 'name': 'movie-review-clf', 'description': 'Tensorflow text classification', 'labels': None, 'created_at': datetime.datetime(2024, 2, 25, 3, 47, 23, tzinfo=datetime.timezone.utc), 'modified_at': datetime.datetime(2024, 2, 25, 3, 47, 23, tzinfo=datetime.timezone.utc)}
{'id': 'learning-code', 'name': 'Code (Tutorial)', 'description': 'Tutorial to add custom code to SAP AI Core', 'labels': None, 'created_at': datetime.datetime(2024, 3, 18, 16, 3, 10, tzinfo=datetime.timezone.utc), 'modified_at': datetime.datetime(2024, 3, 18, 16, 3, 10, tzinfo=datetime.timezone.utc)}
{'id': 'learning-datalines', 'name': 'House Price (Tutorial)', 'description': 'Learning how to ingest data to workflows', 'labels': None, 'created_at': datetime.datetime(2024, 3, 18, 16, 3, 10, tzinfo=datetime.timezone.utc), 'modified_at': datetime.datetime(2024, 3, 18, 16, 3, 10, tzinfo=datetime.timezone.utc)}
{'id': 'learning', 'name': 'Tutorial', 'description': 'Introduction to SAP AI Core', 'labels':

In [29]:
import os
import yaml

# Load environment variables from credentials.yaml
with open('credentials.yaml', 'r') as file:
    env_vars = yaml.safe_load(file)

os.environ['SCENARIO_ID'] = env_vars['SCENARIO_ID']

# List executable with a give scenario
response = ai_core_client.executable.query(scenario_id=os.environ['SCENARIO_ID'], resource_group=os.environ['AICORE_RESOURCE_GROUP'])

for executable in response.resources:
    print(executable.__dict__)


{'id': 'first-pipeline-116', 'scenario_id': 'learning', 'version_id': '2.0', 'name': 'Hello Pipeline', 'description': 'Greets the user', 'deployable': False, 'parameters': [], 'input_artifacts': [], 'output_artifacts': [], 'labels': None, 'created_at': datetime.datetime(2024, 6, 11, 14, 17, 26, tzinfo=datetime.timezone.utc), 'modified_at': datetime.datetime(2024, 6, 11, 17, 15, 7, tzinfo=datetime.timezone.utc)}
{'id': 'first-pipeline-ncellini', 'scenario_id': 'learning', 'version_id': '2.0', 'name': 'Hello Pipeline', 'description': 'Greets the user', 'deployable': False, 'parameters': [], 'input_artifacts': [], 'output_artifacts': [], 'labels': None, 'created_at': datetime.datetime(2024, 3, 18, 16, 3, 10, tzinfo=datetime.timezone.utc), 'modified_at': datetime.datetime(2024, 3, 19, 17, 6, 44, tzinfo=datetime.timezone.utc)}
{'id': 'mfalconaro-first-pipeline', 'scenario_id': 'learning', 'version_id': '1.0', 'name': 'Hello Pipeline', 'description': 'Greets the user', 'deployable': False, '

## Step 8: Run executable

modify the executable ID to the value supplied in your workflow

In [5]:
import os
import yaml

# Load environment variables from credentials.yaml
with open('credentials.yaml', 'r') as file:
    env_vars = yaml.safe_load(file)

os.environ['SCENARIO_ID'] = env_vars['SCENARIO_ID']
os.environ['CONFIGURATION_NAME'] = env_vars['CONFIGURATION_NAME']
os.environ['EXECUTABLE_ID'] = env_vars['EXECUTABLE_ID']

# Create configuration
ai_core_client.configuration.create(
    name=os.environ['CONFIGURATION_NAME'], # custom name of configuration
    scenario_id=os.environ['SCENARIO_ID'], # value from workflow
    executable_id=os.environ['EXECUTABLE_ID'], # value from workflow
    resource_group=os.environ['AICORE_RESOURCE_GROUP']
)

In [7]:
# Lists all configurations
response = ai_core_client.configuration.query(
    resource_group=os.environ['AICORE_RESOURCE_GROUP']
)

for configuration in response.resources:
    print(configuration.__dict__)


{'id': '2f04942b-4036-426e-be98-c20170637563', 'name': 'mfalconaro-hello-pipeline-conf', 'scenario_id': 'learning', 'executable_id': 'mfalconaro-first-pipeline', 'parameter_bindings': [], 'input_artifact_bindings': [], 'created_at': datetime.datetime(2024, 6, 17, 21, 48, 6, tzinfo=datetime.timezone.utc), 'scenario': None}
{'id': '97a2f786-2099-43be-8b99-0ecb3b195149', 'name': 'mfalconaro-hello-pipeline-conf', 'scenario_id': 'learning', 'executable_id': 'mfalconaro-first-pipeline', 'parameter_bindings': [], 'input_artifact_bindings': [], 'created_at': datetime.datetime(2024, 6, 17, 16, 4, 7, tzinfo=datetime.timezone.utc), 'scenario': None}
{'id': '76040917-79ac-4de0-a49f-25a003ec9c29', 'name': 'tomaspf-conf-hello-pipeline', 'scenario_id': 'learning', 'executable_id': 'first-pipeline-116', 'parameter_bindings': [], 'input_artifact_bindings': [], 'created_at': datetime.datetime(2024, 6, 11, 14, 50, 7, tzinfo=datetime.timezone.utc), 'scenario': None}
{'id': 'a6388377-1275-48d2-b582-5c59ec1

In [13]:
import os
import yaml

# Load environment variables from credentials.yaml
with open('credentials.yaml', 'r') as file:
    env_vars = yaml.safe_load(file)

os.environ['CONFIGURATION_ID'] = env_vars['CONFIGURATION_ID']

# Create execution using configuration
response = ai_core_client.execution.create(
    configuration_id=os.environ['CONFIGURATION_ID'],
    resource_group=os.environ['AICORE_RESOURCE_GROUP']
)

response.__dict__

{'id': 'e15f08daad0edb1e',
 'message': 'Execution scheduled',
 'status': <Status.UNKNOWN: 'UNKNOWN'>}

In [14]:
import os
import yaml

# Load environment variables from credentials.yaml
with open('credentials.yaml', 'r') as file:
    env_vars = yaml.safe_load(file)

os.environ['EXECUTION_ID'] = env_vars['EXECUTION_ID']

# Get execution status from SAP AI Core
# execute this multiple times in interval of 30 seconds
response = ai_core_client.execution.get(
    execution_id=os.environ['EXECUTION_ID'], # Change this
    resource_group=os.environ['AICORE_RESOURCE_GROUP']
)

response.__dict__


{'id': 'e15f08daad0edb1e',
 'configuration_id': '2f04942b-4036-426e-be98-c20170637563',
 'configuration_name': 'mfalconaro-hello-pipeline-conf',
 'scenario_id': 'learning',
 'status': <Status.RUNNING: 'RUNNING'>,
 'target_status': <TargetStatus.COMPLETED: 'COMPLETED'>,
 'created_at': datetime.datetime(2024, 6, 17, 22, 7, 43, tzinfo=datetime.timezone.utc),
 'modified_at': datetime.datetime(2024, 6, 17, 22, 8, 59, tzinfo=datetime.timezone.utc),
 'status_message': None,
 'status_details': {'details': [{'container_name': 'init',
    'exit_code': '',
    'last_log_messages': '...rm:linux/amd64,}"time="2024-06-17T22:08:***" level=info msg="Start loading input artifacts..."time="2024-06-17T22:08:***" level=info msg="Alloc=7960 TotalAlloc=11892 Sys=18005 NumGC=3 Goroutines=4"',
    'message': '',
    'pod_name': 'e15f08daad0edb1e-greeter-4102125238',
    'ready': False}],
  'workflow_info': [{'exit_code': '',
    'id': 'e15f08daad0edb1e',
    'message': '',
    'name': 'e15f08daad0edb1e',
    

In [15]:
# Get logs of an execution
response = ai_core_client.execution.query_logs(
    execution_id=os.environ['EXECUTION_ID'],
    resource_group=os.environ['AICORE_RESOURCE_GROUP']
)

for log in response.data.result:
    print(log.__dict__)


{'msg': 'time="2024-06-17T22:08:59.391Z" level=info msg="Starting Workflow Executor" version=v3.5.4', 'timestamp': datetime.datetime(2024, 6, 17, 22, 8, 59, 391476, tzinfo=datetime.timezone.utc)}
{'msg': 'time="2024-06-17T22:08:59.396Z" level=info msg="Using executor retry strategy" Duration=1s Factor=1.6 Jitter=0.5 Steps=5', 'timestamp': datetime.datetime(2024, 6, 17, 22, 8, 59, 396725, tzinfo=datetime.timezone.utc)}
{'msg': 'time="2024-06-17T22:08:59.396Z" level=info msg="Executor initialized" deadline="0001-01-01 00:00:00 +0000 UTC" includeScriptOutput=false namespace=rg-80bbb984-cd927cab podName=e15f08daad0edb1e-greeter-4102125238 templateName=greeter version="&Version{Version:v3.5.4,BuildDate:2024-03-21T09:25:06Z,GitCommit:960af331a8c0a3f2e263c8b90f1daf4303816ba8,GitTag:v3.5.4,GitTreeState:clean,GoVersion:go1.22.1,Compiler:gc,Platform:linux/amd64,}"', 'timestamp': datetime.datetime(2024, 6, 17, 22, 8, 59, 396925, tzinfo=datetime.timezone.utc)}
{'msg': 'time="2024-06-17T22:08:59.51

## Step 9: Update your workflow

*Perform this step over GitHub*. Refer step 9 of the tutorial: https://developers.sap.com/tutorials/ai-core-helloworld.html#87b7f9b0-8420-49bb-a2d5-229a85b330ed



## Step 10: Create new execution

In [ ]:
# Check executable status
response = ai_core_client.executable.query(
    scenario_id = "learning", resource_group='default'
)

for executable in response.resources:
    print(executable.__dict__)


In [ ]:
# Reuse configuration ID to trigger execution
response = ai_core_client.execution.create(
    configuration_id = '<YOUR_CONFIGURATION_ID>', # previously created.
    resource_group = 'default'
)

response.__dict__


In [ ]:
# Get Execution Status
# execute this multiple times in interval of 30 seconds
response = ai_core_client.execution.get(
    execution_id = '<YOUR_EXECUTION_ID>',
    resource_group = 'default'
)

response.__dict__


In [ ]:
# Get Execution Logs
response = ai_core_client.execution.query_logs(
    execution_id = '<YOUR_EXECUTION_ID>',
    resource_group = 'default'
)

for log in response.data.result:
    print(log.__dict__)
